In [123]:
import numpy as np
import cvxpy as cp
from scipy import linalg, integrate, signal
import os

from mujoco import MjModel, MjData
import gymnasium as gym

In [ ]:
filepath = 'pendulum.xml'
model = MjModel.from_xml_path('pendulum.xml')
data = MjData(model)

# 各ボディの質量を取得
for i in range(model.nbody):
    print(f"Body: {model.body(i).name}")
    print(f"  Mass: {model.body_mass[i]}")

# 各ボディの慣性テンソルを取得
for i in range(model.nbody):
    print(f"Body: {model.body(i).name}")
    print(f"  Inertia Tensor: {model.body_inertia[i]}")

シミュレータが持つ設定値を確認したところで、ある程度ラフにモデルを作ってしまいましょう。今回は入出力が複数あるので現代制御的なモデルの表現を用いますが、本質的には古典制御のモデルと変わりません。

In [125]:
M = 1.05 # cart mass
m = 0.5    # pole mass
l = 0.6  # pole length
g = 9.81  # gravitational acceleration
tau = 100 # gear ratio

# system equation
A = np.array([[0,               0, 1, 0], 
              [0,               0, 0, 1],
              [0,          -m*g/M, 0, 0],
              [0, (M + m)*g/(M*l), 0, 0]])   # システム行列
B = np.array([[0], 
              [0],
              [tau/M],
              [-tau/(M*l)]])  

### 最も簡単な制御
 状態フィードバックによる制御を実施します。
 今回の系は多入力多出力系なので現代制御の言葉を用いていますが、要するにPID制御の一種と捉えて大丈夫です。

In [ ]:
filepath = os.path.abspath(filepath)
env = gym.make('InvertedPendulum-v5', xml_file=filepath, render_mode="human")

# パラメータ設定
x_ref = np.zeros(4)  # 目標値
K_p = -np.array([5, 1])     # 制御ゲイン
K_d = -np.array([1, 1])
K = np.hstack((K_p, K_d))
print(K)

# PID controller
def pid_control(state, ref, K):
    x = state
    error = ref - x
    u = K[:2] @ error[:2]  + K[2:] @ error[2:]

    return np.array([u])


state, info = env.reset()
print(state)
for t in range(300):
    action = pid_control(state, x_ref, K)
    next_state, reward, done, truncated, info = env.step(action)
    state = next_state
    
    # if done:
    #     break
env.close()

手で調整するのは難しそうなので、LQRを利用してゲインを決めます

In [ ]:
filepath = os.path.abspath(filepath)
env = gym.make('InvertedPendulum-v5', xml_file=filepath, render_mode="human", reset_noise_scale=0.0)

# 重み行列 Q, R（任意に設定）
Q = np.diag([1, 1, 1, 1])       # 状態の重み（正定値行列）
R = np.array([[100]])        # 入力の重み（正定値行列）

# リカッチ方程式を解いてLQRゲインを計算
X = np.matrix(linalg.solve_continuous_are(A, B, Q, R))
KK = np.squeeze(np.array(linalg.inv(R) * (B.T @ X)))
print(KK)

state, info = env.reset()
dt = 0.02

for k in range(500):
    x_ref[0] = 0.8*np.sin(2*np.pi*k*dt/2)
    action = pid_control(state, x_ref, KK)
    next_state, reward, done, truncated, info = env.step(action)
    state = next_state
    
    # if done:
    #     break
env.close()

### IFT
IFTを使ったチューニングもやってみましょう。今回はFRITを試してみます。

### MPC(モデル予測制御)
今回はFA分野でも導入の試みがあるPFCの例を示します。以下の実装は最も簡単な例です。実装者にとってはあまり簡単ではないですが、通常のMPCと比べるとオンライン計算の量が非常に少ないためPLCでも実装例があるそうです。また、利用者にとってはパラメータ調整が簡単になるメリットがあります。
IHIとか住友化学とかで例あり。

In [ ]:
# 環境の設定
env = gym.make('InvertedPendulum-v5', xml_file=filepath, render_mode="human", reset_noise_scale=0.0)

# パラメータ設定
T_ctrl = np.array([0.1, 0.1]) # 時定数
h1 = 2 # receding horizon 1
h2 = 12 # receding horizon 2
u_min, u_max = -3, 3 # input constraint


# prepare discrete system
dt = 0.02
C = np.array([[1, 0, 0, 0],
              [0, 1, 0, 0]])
D = np.array([[0]])
d_system = signal.cont2discrete((A - B@np.array([KK]), B, C, D), dt) # include state feedback to stabilize plant model 
A_d, B_d, C_d, D_d, dt = d_system

# offline calculation for pfc
G_h1 = np.hstack([C_d@np.linalg.matrix_power(A_d, i)@B_d for i in reversed(range(h1 + 1))]) # coeff for horizon 1
step1_array = np.ones(h1 + 1).T
lamp1_array = np.array([i*dt for i in range(h1 + 1)]).T
G_h2 = np.hstack([C_d@np.linalg.matrix_power(A_d, i)@B_d for i in reversed(range(h2 + 1))]) # coeff for horizon 2
step2_array = np.ones(h2 + 1).T
lamp2_array = np.array([i*dt for i in range(h2 + 1)]).T

S_f_mat = np.vstack((np.hstack([np.dot(G_h1, step1_array), np.dot(G_h2, step2_array)]),
                    np.hstack([np.dot(G_h1, lamp1_array), np.dot(G_h2, lamp2_array)]))).T
def fpc_control(y_ideal, x):
    u_delta = np.linalg.pinv(S_f_mat)@(y_ideal - np.hstack([C_d@np.linalg.matrix_power(A_d, h1)@x, C_d@np.linalg.matrix_power(A_d, h2)@x]) + np.hstack((C_d@x, C_d@x)))

    return u_delta


import time
state, info = env.reset()
action = np.array([0])
start_time = time.time()
max_delta = 0
ref = np.zeros(4)
# TODO set input to pid control
for k in range(500):
    x_ref[0] = 0.8*np.sin(2*np.pi*k*dt/2)
    y_ideal = np.hstack([-(1 - np.exp(-h1*dt/T_ctrl))*(state[:2] - x_ref[:2]), -(1 - np.exp(-h2*dt/T_ctrl))*(state[:2] - x_ref[:2])]) # this is ideal offset with respect to current value
    delta = fpc_control(y_ideal, state)
    max_delta = max(max_delta, delta[0])
    action = np.array([delta[0]]) + pid_control(state, ref, KK)
    action = np.clip(action, u_min, u_max)
    next_state, reward, done, truncated, info = env.step(action)
    state = next_state

    if done:
        break
print("elapsed time", time.time() - start_time)
print(max_delta)
env.close()


### 強化学習
今回は学習済みのパラメータを利用します。全面的にライブラリの力を借りるとなんの知識もいらないので楽ちんですね

In [7]:
import gymnasium as gym
import os
from stable_baselines3 import PPO

filepath = 'pendulum.xml'
filepath = os.path.abspath(filepath)

save_dir = "models/ppo_inverted_pendulum"
final_model_path = os.path.join(save_dir, "ppo_inverted_pendulum_final")

model = PPO.load(final_model_path)
env = gym.make('InvertedPendulum-v5', xml_file=filepath, render_mode="human", reset_noise_scale=0.3)
obs, _ = env.reset()

for _ in range(600):  # テストエピソードの長さ
    action, _ = model.predict(obs, deterministic=True)
    obs, reward, terminated, truncated, info = env.step(action)
    if terminated or truncated:
        obs, _ = env.reset()

env.close()